In [19]:
import numpy as np
import pandas as pd
import os
import math

In [44]:
pd.DataFrame([[1,2,3],[3,4,5]])[[0,1]].mean(axis=1)

0    1.5
1    3.5
dtype: float64

In [45]:
# this function should be applied to a df containing the data for matches (aggregated or not) in the initial 
# clean format that we defined
# THIS FUNCTION ORDERS DISTANCES AND CLUSTERS FROM HIGHGEST TO LOWEST
def aggData(df):
    distances = df.iloc[:, 6:12]
    distances.fillna(-999)
    centroids = df.iloc[:, 12:16]
    centroids.fillna(-999)
    clusters = df.iloc[:, 16:20].values
    new_distances = np.sort(distances.values, axis=1)[:,::-1]
    new_centroids = np.sort(centroids.values, axis=1)[:,::-1]
    new_clusters = np.sort(clusters, axis=1)[:,::-1]
    distances = distances.replace(-999, np.nan)
    centroids = centroids.replace(-999, np.nan)
    df.iloc[:, 6:12] = new_distances
    df.iloc[:, 12:16] = new_centroids
    df.iloc[:, 16:20] = new_clusters
    df['agility']=np.where(df['strategy']!=df['strategy'].shift(-1), 1,0)
    ranking = df['ranking']
    df = df.drop('ranking', axis=1)
    df.insert(len(df.columns), 'ranking', ranking)
    df['cluster_A'] = df['cluster_A'] / df['n_alive']
    df['cluster_B'] = df['cluster_B'] / df['n_alive']
    df['cluster_C'] = df['cluster_C'] / df['n_alive']
    df['cluster_D'] = df['cluster_D'] / df['n_alive']
    df = df[df.n_alive > 0]
    df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] / 100
    df.loc[:, ['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:, ['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']].applymap(lambda x:np.log(x) if x != 0 else 0)
    group = df[['matchId','time','distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']].groupby(['matchId','time']).mean().reset_index()
    group.columns = ['matchId','time','d12','d13','d14','d23', 'd24', 'd34','dc1', 'dc2', 'dc3', 'dc4']
    group['d'] = group[['d12','d13','d14','d23', 'd24', 'd34']].mean(axis=1)
    group['dc'] = group[['dc1', 'dc2', 'dc3', 'dc4']].mean(axis=1)
    group = group.drop(['d12','d13','d14','d23', 'd24', 'd34','dc1', 'dc2', 'dc3', 'dc4'], axis=1)
    df = pd.merge(df, group, how = 'left', on=['matchId','time'])
    df['distance1_2'] = df['distance1_2'] / df['d']
    df['distance1_3'] = df['distance1_3'] / df['d']
    df['distance1_4'] = df['distance1_4'] / df['d']
    df['distance2_3'] = df['distance2_3'] / df['d']
    df['distance2_4'] = df['distance2_4'] / df['d']
    df['distance3_4'] = df['distance3_4'] / df['d']
    df['distance_centroid1'] = df['distance_centroid1'] / df['dc']
    df['distance_centroid2'] = df['distance_centroid2'] / df['dc']
    df['distance_centroid3'] = df['distance_centroid3'] / df['dc']
    df['distance_centroid4'] = df['distance_centroid4'] / df['dc']
    df = df.drop(['d','dc'], axis=1)
    df = df.drop(['time','n_alive','in_aircraft'], axis=1)
    means = df.groupby(['matchId', 'teamId']).mean().reset_index()
    stds = df.groupby(['matchId', 'teamId']).std().reset_index()
    means = means.fillna(99)
    means = means.replace(np.inf, 0)
    stds = stds.fillna(0)
    new_df = pd.merge(stds, means, how = 'left', on=['matchId', 'teamId'])
    new_df = new_df.drop(['n_players_x', 'ranking_x', 'agility_x'], axis=1)
    return new_df

In [58]:
# THIS FUNCTION ORDERS DISTANCES AND CLUSTERS FROM HIGHGEST TO LOWEST
def aggDataNoLog(df):
    distances = df.iloc[:, 6:12]
    distances.fillna(-999)
    centroids = df.iloc[:, 12:16]
    centroids.fillna(-999)
    clusters = df.iloc[:, 16:20].values
    new_distances = np.sort(distances.values, axis=1)[:,::-1]
    new_centroids = np.sort(centroids.values, axis=1)[:,::-1]
    new_clusters = np.sort(clusters, axis=1)[:,::-1]
    distances = distances.replace(-999, np.nan)
    centroids = centroids.replace(-999, np.nan)
    df.iloc[:, 6:12] = new_distances
    df.iloc[:, 12:16] = new_centroids
    df.iloc[:, 16:20] = new_clusters
    df['agility']=np.where(df['strategy']!=df['strategy'].shift(-1), 1,0)
    ranking = df['ranking']
    df = df.drop('ranking', axis=1)
    df.insert(len(df.columns), 'ranking', ranking)
    df['cluster_A'] = df['cluster_A'] / df['n_alive']
    df['cluster_B'] = df['cluster_B'] / df['n_alive']
    df['cluster_C'] = df['cluster_C'] / df['n_alive']
    df['cluster_D'] = df['cluster_D'] / df['n_alive']
    df = df[df.n_alive > 0]
    df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] / 100
    group = df[['matchId','time','distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']].groupby(['matchId','time']).mean().reset_index()
    group.columns = ['matchId','time','d12','d13','d14','d23', 'd24', 'd34','dc1', 'dc2', 'dc3', 'dc4']
    group['d'] = group[['d12','d13','d14','d23', 'd24', 'd34']].mean(axis=1)
    group['dc'] = group[['dc1', 'dc2', 'dc3', 'dc4']].mean(axis=1)
    group = group.drop(['d12','d13','d14','d23', 'd24', 'd34','dc1', 'dc2', 'dc3', 'dc4'], axis=1)
    df = pd.merge(df, group, how = 'left', on=['matchId','time'])
    df['distance1_2'] = df['distance1_2'] / df['d']
    df['distance1_3'] = df['distance1_3'] / df['d']
    df['distance1_4'] = df['distance1_4'] / df['d']
    df['distance2_3'] = df['distance2_3'] / df['d']
    df['distance2_4'] = df['distance2_4'] / df['d']
    df['distance3_4'] = df['distance3_4'] / df['d']
    df['distance_centroid1'] = df['distance_centroid1'] / df['dc']
    df['distance_centroid2'] = df['distance_centroid2'] / df['dc']
    df['distance_centroid3'] = df['distance_centroid3'] / df['dc']
    df['distance_centroid4'] = df['distance_centroid4'] / df['dc']
    df = df.drop(['d','dc'], axis=1)
    df = df.drop(['time','n_alive','in_aircraft'], axis=1)
    means = df.groupby(['matchId', 'teamId']).mean().reset_index()
    stds = df.groupby(['matchId', 'teamId']).std().reset_index()
    means = means.fillna(99)
    means = means.replace(np.inf, 0)
    stds = stds.fillna(0)
    new_df = pd.merge(stds, means, how = 'left', on=['matchId', 'teamId'])
    new_df = new_df.drop(['n_players_x', 'ranking_x', 'agility_x'], axis=1)
    return new_df

In [3]:
# THIS FUNCTION DOES NOT ORDER DISTANCES, ONLY CLUSTERS
def aggDataUnordered(df):
    clusters = df.iloc[:, 16:20].values
    new_clusters = np.sort(clusters, axis=1)[:,::-1]
    df.iloc[:, 16:20] = new_clusters
    df = df.replace(-999, np.nan)
    df['agility']=np.where(df['strategy']!=df['strategy'].shift(-1), 1,0)
    ranking = df['ranking']
    df = df.drop('ranking', axis=1)
    df.insert(len(df.columns), 'ranking', ranking)
    df['cluster_A'] = df['cluster_A'] / df['n_alive']
    df['cluster_B'] = df['cluster_B'] / df['n_alive']
    df['cluster_C'] = df['cluster_C'] / df['n_alive']
    df['cluster_D'] = df['cluster_D'] / df['n_alive']
    df = df[df.n_alive > 0]
    df = df.drop(['time','n_alive','in_aircraft'], axis=1)
    df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] / 100
    df.loc[:, ['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:, ['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']].applymap(lambda x:np.log(x) if x != 0 else 0)
    means = df.groupby(['matchId', 'teamId']).mean().reset_index()
    stds = df.groupby(['matchId', 'teamId']).std().reset_index()
    means = means.fillna(99)
    means = means.replace(np.inf, 0)
    stds = stds.fillna(0)
    new_df = pd.merge(stds, means, how = 'left', on=['matchId', 'teamId'])
    new_df = new_df.drop(['n_players_x', 'ranking_x', 'agility_x'], axis=1)
    return new_df

In [ ]:
# THIS FUNCTION DOES NOT ORDER DISTANCES, ONLY CLUSTERS
def aggDataUnorderedNoLog(df):
    clusters = df.iloc[:, 16:20].values
    new_clusters = np.sort(clusters, axis=1)[:,::-1]
    df.iloc[:, 16:20] = new_clusters
    df = df.replace(-999, np.nan)
    df['agility']=np.where(df['strategy']!=df['strategy'].shift(-1), 1,0)
    ranking = df['ranking']
    df = df.drop('ranking', axis=1)
    df.insert(len(df.columns), 'ranking', ranking)
    df['cluster_A'] = df['cluster_A'] / df['n_alive']
    df['cluster_B'] = df['cluster_B'] / df['n_alive']
    df['cluster_C'] = df['cluster_C'] / df['n_alive']
    df['cluster_D'] = df['cluster_D'] / df['n_alive']
    df = df[df.n_alive > 0]
    df = df.drop(['time','n_alive','in_aircraft'], axis=1)
    df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
              'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] = df.loc[:,['distance1_2','distance1_3','distance1_4','distance2_3', 'distance2_4', 'distance3_4',
                                                                                                                   'distance_centroid1', 'distance_centroid2', 'distance_centroid3', 'distance_centroid4']] / 100
    means = df.groupby(['matchId', 'teamId']).mean().reset_index()
    stds = df.groupby(['matchId', 'teamId']).std().reset_index()
    means = means.fillna(99)
    means = means.replace(np.inf, 0)
    stds = stds.fillna(0)
    new_df = pd.merge(stds, means, how = 'left', on=['matchId', 'teamId'])
    new_df = new_df.drop(['n_players_x', 'ranking_x', 'agility_x'], axis=1)
    return new_df

In [59]:
#eg.
a = 0
for file in os.listdir('../combined_csvs'):
    df = pd.read_csv(f'../combined_csvs/{file}')
    new_df = aggData(df)
    if a == 0:
        all_df = new_df
        a = a+1
    else:
        all_df = pd.concat([all_df, new_df])
    
# aggregated_df = aggData(df)
#
# analysis for ML models should be ran in the resulting dataframe format
# MISSING: merge risk behaviour information

In [60]:
all_df = all_df.reset_index(drop=True)

In [61]:
all_df.to_csv('../final_csvs/all_df_ordered_no_log.csv')

In [63]:
risk_df = pd.read_csv('D:/PUBG/risk_csv.csv')
risk_df.teamId = risk_df.teamId.astype('int')

In [64]:
all_risk_df = pd.merge(all_df, risk_df, how='left', on=['matchId','teamId'])

In [65]:
all_risk_df.to_csv('../final_csvs/all_risk_df_ordered_no_log.csv')